In [1]:
# ----------------------------------------------------------------------
# Numenta Platform for Intelligent Computing (NuPIC)
# Copyright (C) 2019, Numenta, Inc.  Unless you have an agreement
# with Numenta, Inc., for a separate license for this software code, the
# following terms and conditions apply:
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero Public License version 3 as
# published by the Free Software Foundation.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Affero Public License for more details.
#
# You should have received a copy of the GNU Affero Public License
# along with this program.  If not, see http://www.gnu.org/licenses.
#
# http://numenta.org/licenses/
# ----------------------------------------------------------------------

In [2]:
# Uncomment the following line to install nupic.torch
!pip install -e git+https://github.com/numenta/nupic.torch.git#egg=nupic.torch

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.manual_seed(18)
np.random.seed(18)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class RandomNoise(object):
  """
  An image transform that adds noise to random pixels in the image.
  """
  def __init__(self, noiselevel=0.0, whiteValue=0.1307 + 2*0.3081):
    """
    :param noiselevel:
      From 0 to 1. For each pixel, set its value to whiteValue with this
      probability. Suggested whiteVolue is 'mean + 2*stdev'
    """
    self.noiseLevel = noiselevel
    self.whiteValue = whiteValue

  def __call__(self, image):
    a = image.view(-1)
    numNoiseBits = int(a.shape[0] * self.noiseLevel)
    noise = np.random.permutation(a.shape[0])[0:numNoiseBits]
    a[noise] = self.whiteValue
    return image

def train(model, loader, optimizer, criterion):
    """
    Train the model using given dataset loader. 
    Called on every epoch.
    :param model: pytorch model to be trained
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param optimizer: Optimizer object used to train the model.
    :type optimizer: :class:`torch.optim.Optimizer`
    :param criterion: loss function to use
    :type criterion: function
    """
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(model, loader, criterion):
    """
    Evaluate pre-trained model using given dataset loader.
    Called on every epoch.
    :param model: Pretrained pytorch model
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param criterion: loss function to use
    :type criterion: function
    :return: Dict with "accuracy", "loss" and "total_correct"
    """
    model.eval()
    loss = 0
    total_correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            total_correct += pred.eq(target.view_as(pred)).sum().item()
    
    return {"accuracy": total_correct / len(loader.dataset), 
            "loss": loss / len(loader.dataset), 
            "total_correct": total_correct}

### Parameters

In [5]:
# MLP configuration
INPUT_SIZE = 28*28  
HIDDEN_SIZE = 1000  
OUTPUT_SIZE = 10    

# Sparsity parameters
WEIGHT_SPARSITY = 0.3

# K-Winners parameters
K = 100
K_INFERENCE_FACTOR = 1.5
BOOST_STRENGTH = 1.0
BOOST_STRENGTH_FACTOR = 0.9

# Training parameters
LEARNING_RATE = 0.01
MOMENTUM = 0.5
EPOCHS = 10
FIRST_EPOCH_BATCH_SIZE = 4
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000

# Noise test parameters
NOISE_LEVELS = [0.05, 0.10, 0.15, 0.20, 0.25]

###  Sparse Linear Model
Create a sparse MLP network with 2 hidden layers. We apply 30% sparsity to every hidden layer and use k-winners activation between the layers

In [6]:
from nupic.torch.modules import KWinners, SparseWeights, Flatten

sparseNN = nn.Sequential(
    Flatten(),

    # First Hidden layer
    SparseWeights(nn.Linear(INPUT_SIZE, HIDDEN_SIZE), weightSparsity=WEIGHT_SPARSITY),
    KWinners(n=HIDDEN_SIZE, k=K, kInferenceFactor=K_INFERENCE_FACTOR, 
             boostStrength=BOOST_STRENGTH, boostStrengthFactor=BOOST_STRENGTH_FACTOR),

    # Second Hidden layer
    SparseWeights(nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE), weightSparsity=WEIGHT_SPARSITY),
    KWinners(n=HIDDEN_SIZE, k=K, kInferenceFactor=K_INFERENCE_FACTOR, 
             boostStrength=BOOST_STRENGTH, boostStrengthFactor=BOOST_STRENGTH_FACTOR),

    # Output layer
    nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE),
    nn.LogSoftmax(dim=1)
).to(device)

### Load MNIST Dataset

In [7]:
normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=normalize)
test_dataset = datasets.MNIST('data', train=False, transform=normalize)

# Configure data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)
first_loader = torch.utils.data.DataLoader(train_dataset, batch_size=FIRST_EPOCH_BATCH_SIZE, shuffle=True)

### Train
On the first epoch we use smaller batch size to calculate the duty cycles used by the k-winner function. Once the duty cycles stabilize we can use larger batch sizes.

In [8]:
sgd = optim.SGD(sparseNN.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
train(model=sparseNN, loader=first_loader, optimizer=sgd, criterion=F.nll_loss)
results = test(model=sparseNN, loader=test_loader, criterion=F.nll_loss)
print(results)

{'accuracy': 0.9506, 'loss': 0.16315889587402344, 'total_correct': 9506}


At this point the duty cycles should be stable and we can train on larger batch sizes

In [9]:
for epoch in range(1, EPOCHS):
    train(model=sparseNN, loader=train_loader, optimizer=sgd, criterion=F.nll_loss)
    results = test(model=sparseNN, loader=test_loader, criterion=F.nll_loss)
    print(results)

{'accuracy': 0.9627, 'loss': 0.12201591415405273, 'total_correct': 9627}
{'accuracy': 0.9634, 'loss': 0.12190820617675781, 'total_correct': 9634}
{'accuracy': 0.9623, 'loss': 0.12345575714111329, 'total_correct': 9623}
{'accuracy': 0.9639, 'loss': 0.1185587173461914, 'total_correct': 9639}
{'accuracy': 0.9611, 'loss': 0.11994301071166992, 'total_correct': 9611}
{'accuracy': 0.9633, 'loss': 0.11600606689453125, 'total_correct': 9633}
{'accuracy': 0.9634, 'loss': 0.11699238586425781, 'total_correct': 9634}
{'accuracy': 0.9639, 'loss': 0.11530724716186523, 'total_correct': 9639}
{'accuracy': 0.9633, 'loss': 0.11797227020263672, 'total_correct': 9633}


### Noise
Add noise to the input and check the test accuracy

In [10]:
for noise in NOISE_LEVELS:
    noise_transform = transforms.Compose([transforms.ToTensor(), RandomNoise(noise), 
                                      transforms.Normalize((0.1307,), (0.3081,))])
    noise_dataset = datasets.MNIST('data', train=False, transform=noise_transform)
    noise_loader = torch.utils.data.DataLoader(noise_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)

    results = test(model=sparseNN, loader=noise_loader, criterion=F.nll_loss)
    print(noise, ":", results)

0.05 : {'accuracy': 0.9541, 'loss': 0.14802040100097658, 'total_correct': 9541}
0.1 : {'accuracy': 0.939, 'loss': 0.1916733642578125, 'total_correct': 9390}
0.15 : {'accuracy': 0.9094, 'loss': 0.2852293640136719, 'total_correct': 9094}
0.2 : {'accuracy': 0.8639, 'loss': 0.4125948455810547, 'total_correct': 8639}
0.25 : {'accuracy': 0.8043, 'loss': 0.5801907653808593, 'total_correct': 8043}
